In [1]:
import sklearn
import os
import json
import pandas as pd

In [2]:
df_test = pd.read_csv('renttherunway-dataset/data/dataset_split/test.csv')

In [3]:
f_paths=[]
for r,d,f in os.walk('final_preds'):
     for f_name in f:
            if not 'checkpoint' in f_name:
                f_paths.append(f_name)
f_paths

['knn_euclidean.json',
 'demographic_result.json',
 'knn_manhattan.json',
 'recommendations_collaborative_filtering.json',
 'baseline.json',
 'knn_cosine.json',
 'tfidf_results_top_100.json']

In [4]:
results = []

for f_path in f_paths:
    with open(f'final_preds/{f_path}') as f_handle:
        results.append(json.load(f_handle))

In [6]:
Ns = [2, 5, 10, 20, 30, 40]
for top_n in Ns:
    acc_results = []
    rec_results = []
    for ds_idx, result in enumerate(results):
        acc_all = []
        rec_all = []
        for user, preds in list(result.items()):
            user_transactions = df_test[df_test.user_id == int(user)]
            preds = list(sorted(preds.items(), key=lambda item: item[1], reverse=True))[:top_n]
            pred_inx, pred_vals = zip(*preds)
            pred_inx = list(map(lambda x : int(x), pred_inx))

            recommended_and_relevant = user_transactions[user_transactions.item_id.isin(pred_inx)]

            acc = len(recommended_and_relevant)/len(user_transactions)

            recommended_rent_ones = recommended_and_relevant[recommended_and_relevant.class_rating==1]
            rent_ones = user_transactions[user_transactions.class_rating==1]

            if len(rent_ones) == 0:
                rec = 0
            else:
                rec = len(recommended_rent_ones)/len(rent_ones)

            acc_all.append(acc)
            rec_all.append(rec)

        acc_avg = sum(acc_all)/ len(acc_all)
        rec_avg = sum(rec_all)/ len(rec_all)
        acc_results.append(acc_avg)
        rec_results.append(rec_avg)
        print(f'Dataset: {f_paths[ds_idx]}')
        print(f'ACC: {acc_avg}')
        print(f'REC: {rec_avg}')

    df_results = pd.DataFrame({"Dataset":f_paths,"Acc": acc_results,"Rec": rec_results})
    df_results.to_csv(f'results/final_results_top_{top_n}.csv', sep='|')

Dataset: knn_euclidean.json
ACC: 0.005856187262686996
REC: 0.004735689338779376
Dataset: demographic_result.json
ACC: 0.0015982951518380393
REC: 0.0010655301012253596
Dataset: knn_manhattan.json
ACC: 0.004264895222873379
REC: 0.0037626531699520506
Dataset: recommendations_collaborative_filtering.json
ACC: 0.004350914580003552
REC: 0.0037293553542887587
Dataset: baseline.json
ACC: 0.008257858284496537
REC: 0.007310720416740662
Dataset: knn_cosine.json
ACC: 0.004617297105309891
REC: 0.004262120404901438
Dataset: tfidf_results_top_100.json
ACC: 0.007517906825312259
REC: 0.007170129639495649
Dataset: knn_euclidean.json
ACC: 0.013510060168624364
REC: 0.012227697863020184
Dataset: demographic_result.json
ACC: 0.00349257088734979
REC: 0.0025750310779612856
Dataset: knn_manhattan.json
ACC: 0.015657610717879765
REC: 0.013679694040642362
Dataset: recommendations_collaborative_filtering.json
ACC: 0.008701829160007104
REC: 0.007902681584088083
Dataset: baseline.json
ACC: 0.01698928550287101
REC: 0